In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
import numpy as np

# Read CSV file
data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data = data.drop(["ID","Age","ZIP Code"],axis = 1)

# Split the data into features (X) and labels (y)
X = data.drop('Personal Loan', axis=1).values
y = data['Personal Loan'].values


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        #self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(5, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        #x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

net = Net()

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# Define particle swarm optimization
def pso(n_particles, n_iterations, c1, c2):
    # Initialize particle positions and velocities
    positions = []
    velocities = []
    for i in range(n_particles):
        position = []
        velocity = []
        for param in net.parameters():
            position.append(np.random.rand(*param.shape))
            velocity.append(np.zeros(param.shape))
        positions.append(position)
        velocities.append(velocity)

    # Initialize personal best positions and global best position
    pbest_positions = positions.copy()
    pbest_fitness = [float('inf')] * n_particles
    gbest_position = None
    gbest_fitness = float('inf')

    for i in range(n_iterations):
        # Evaluate fitness of particles
        fitnesses = []
        for position in positions:
            with torch.no_grad():
                for param, value in zip(net.parameters(), position):
                    param.copy_(torch.from_numpy(value))
            outputs = net(torch.from_numpy(X_train).float())
            loss = criterion(outputs.squeeze(), torch.from_numpy(y_train).float())
            fitnesses.append(loss.item())

        # Update personal best positions and global best position
        for j in range(n_particles):
            if fitnesses[j] < pbest_fitness[j]:
                pbest_fitness[j] = fitnesses[j]
                pbest_positions[j] = positions[j]
            if fitnesses[j] < gbest_fitness:
                gbest_fitness = fitnesses[j]
                gbest_position = positions[j]

        # Update velocities and positions of particles
        for j in range(n_particles):
            for k in range(len(velocities[j])):
                velocities[j][k] = w * velocities[j][k] + c1 * np.random.rand() * (pbest_positions[j][k] - positions[j][k]) + c2 * np.random.rand() * (gbest_position[k] - positions[j][k])
                positions[j][k] += velocities[j][k]

    return gbest_position

# Train neural network using particle swarm optimization
gbest_position = pso(n_particles=10, n_iterations=1000, c1=2.0, c2=2.0)
with torch.no_grad():
    for param, value in zip(net.parameters(), gbest_position):
        param.copy_(torch.from_numpy(value))

# Evaluate accuracy on test set
outputs = net(torch.from_numpy(X_test).float())
predicted = (outputs.squeeze() > 0.5).float()
accuracy = (predicted == torch.from_numpy(y_test).float()).sum().item() / len(y_test)
print('Accuracy:', accuracy)


Accuracy: 0.83
